In [1]:
# http://flask.pocoo.org/docs/0.12/tutorial/templates/


from flask import Flask, request, render_template, url_for
import requests, json
import time
app = Flask(__name__)
host_url = "http://178.162.214.41:5000/"
main_url = 'http://127.0.0.1:8332/rest/'

@app.route('/')
def hello_world():
    return render_template("index.html")
#     return "<a href='./info'>zBlock Explorer<a>"


@app.route('/tx/<transaction_id>')
def transaction(transaction_id):
    a = time.time()
#     transaction_id = "42f9df54a39026ccb54362141c41713968f19e1f14949ab6609b03ffa4b7f120"
    url = main_url + "tx/" + transaction_id +'.json'
    tx_response = requests.get(url)

    content = "<h2>Transaction</h2><hr/>"
    content += str(time.time()-a) + " (s) Elapsed Time!<br/><hr/>"
    content += "<table width='100%'>"

    result = json.loads(tx_response.content)
    for key, value in result.items():
        if (key in ["txid","hash"]):
            content += ('<tr><td>{:25}</td><td>{:20}</td><td>{:_<5}</td><td><a href="{}tx/{}">{}</a></td></td>'.format(key, str(type(value)), len(str(value)), request.host_url, str(value), str(value)))

        elif (key in ["vin","vout"] ):
            content +=("<tr><td>------ "+ key + " ------ </td></tr>")
            for vin in value:
                for key2, value2 in vin.items():
                    if (key2 in ["txid"]):
                        content += ('<tr><td>{:25}</td><td>{:20}</td><td>{:_<5}</td><td><a href="{}tx/{}">{}</a></td></td>'.format(key2, str(type(value2)), len(str(value2)), request.host_url, str(value2), str(value2)))
                    else:
                        content +=('<tr><td>{:25}</td><td>{:20}</td><td>{:_<5}</td><td title="{}">{}</td></tr>'.format(key2, str(type(value2)), len(str(value2)), str(value2),str(value2)[:60]))
            content +=("<tr><td>------ "+ key + " ------ </td></tr>")
        elif (key in ["blockhash"]):
            content +=('<tr><td>{:25}</td><td>{:20}</td><td>{:_<5}</td><td><a href="{}block/{}">{}</a></td></tr>'.format(key, str(type(value)), len(str(value)), request.host_url, str(value) ,str(value)))
        else:
            content +=('<tr><td>{:25}</td><td>{:20}</td><td>{:_<5}</td><td>{}</td></tr>'.format(key, str(type(value)), len(str(value)), str(value)[:60]))

    
    content += "</table>"
    content += "<hr/><a href='../info'>zBlock Explorer<a>"

    return content


@app.route('/info')
def info():
    url = main_url + 'chaininfo.json'
    chain_response = requests.get(url)
    result = json.loads(chain_response.content)

    content = "<h2>zBlock Explorer</h2><hr/><table width='100%'>"
    for key, value in result.items():
        if(key in "bestblockhash"):
            content +='<tr><td>{:20}</td><td>{:20}</td><td>{:_<5}</td><td><a href="{}">{}</a></td></tr>'.format(key, str(type(value)), len(str(value)), request.host_url + "block/" + str(value), str(value))
        else:
            content +='<tr><td>{:20}</td><td>{:20}</td><td>{:_<5}</td><td>{}</td></tr>'.format(key, str(type(value)), len(str(value)), str(value)[:70])
    content += "</table><hr/><a href='./peers'>Peers Count</a> | <a href='./genesis'>Genesis</a> | <a href=./block/"+ result["bestblockhash"]+">Last Block</a>"
    return content

@app.route('/peers')
def peers():
    import pandas as pd
    df = pd.read_csv("peers_db")
    content = "<h2>Peers</h2><hr/>"
    content += "Count visited peers until 06 March 2018 - 18:00 <b><h1>" + str(len(df)) + "</h1></b><hr/>"
    content += "<a href='./info'>zBlock Explorer<a>"

    return content

@app.route('/genesis')
def genesis():
    from flask import Flask,redirect
    return redirect("./block/000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f", code=302)


@app.route('/block/<block_hash>')
def getblock(block_hash):
    a = time.time()
    url = main_url + "block/" + block_hash +'.json'
    block_response = requests.get(url)

    content = "<style>{}</style>".format(open("explorer.css", "r").read())
    
    content += "<h2>Block</h2><hr/>"
    content += str(time.time()-a) + " (s) Elapsed Time!<br/><hr/>"
    content += "<table width='100%'>"

    result = json.loads(block_response.content)
    for key, value in result.items():
        if(key in ["hash","nextblockhash","previousblockhash"]):
            content += '<tr><td>{:25}</td><td>{:20}</td><td>{:_<5}</td><td><a href="{}">{}</a></td></tr>'.format(key, str(type(value)), len(str(value)), request.host_url +"block/" + str(value) ,str(value))
        else:
            content += '<tr><td>{:25}</td><td>{:20}</td><td>{:_<5}</td><td>{}</td></tr>'.format(key, str(type(value)), len(str(value)), str(value)[:60])
    
    content += "</table><hr/>"
    content += str(len(result["tx"])) + " Transaction(s) in the block: <div class='tx_page'>"

    for i in range (min(20,len(result["tx"]))):
        content += "[<a href=/tx/{}>{}</a>],".format(result["tx"][i]["txid"],str(i))
    if(len(result["tx"])>20):
        content += "..."
    content += "</div><hr/><a href='../info'>zBlock Explorer<a>"
    
    return content

In [2]:
#https://themefisher.com/


app.run(host="0.0.0.0", port="5001")

 * Running on http://0.0.0.0:5001/ (Press CTRL+C to quit)
5.123.246.18 - - [30/Mar/2018 10:31:18] "GET / HTTP/1.1" 200 -
5.123.246.18 - - [30/Mar/2018 10:31:19] "GET /static/css/font-awesome.min.css HTTP/1.1" 200 -
5.123.246.18 - - [30/Mar/2018 10:31:19] "GET /static/css/bootstrap.min.css HTTP/1.1" 200 -
5.123.246.18 - - [30/Mar/2018 10:31:20] "GET /static/css/owl.carousel.css HTTP/1.1" 200 -
5.123.246.18 - - [30/Mar/2018 10:31:20] "GET /static/css/et-font.css HTTP/1.1" 200 -
5.123.246.18 - - [30/Mar/2018 10:31:20] "GET /static/css/justifiedGallery.min.css HTTP/1.1" 200 -
5.123.246.18 - - [30/Mar/2018 10:31:20] "GET /static/css/animate.css HTTP/1.1" 200 -
5.123.246.18 - - [30/Mar/2018 10:31:20] "GET /static/css/owl.theme.css HTTP/1.1" 200 -
5.123.246.18 - - [30/Mar/2018 10:31:22] "GET /static/css/magnific-popup.css HTTP/1.1" 200 -
5.123.246.18 - - [30/Mar/2018 10:31:22] "GET /static/css/main.css HTTP/1.1" 200 -
5.123.246.18 - - [30/Mar/2018 10:31:22] "GET /static/js/modernizr-custom.js